In [1]:
!hdfs dfs -rmr /user/dominospark/mypi/

rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted /user/dominospark/mypi


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType
import random

In [3]:
sparkSession = SparkSession.builder.appName("Calculate Pi") \
.config("spark.dynamicAllocation.enabled", "false") \
.getOrCreate()
sc=sparkSession.sparkContext

In [4]:
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1


In [5]:


columns = StructType([ StructField("name", StringType(), True),
                      StructField("value", DoubleType(), True)
                    ])

count = sc.parallelize(range(0, 1000),1) \
             .filter(inside).count()
data = [("Pi",4.0 * count/1000)]

df = sparkSession.createDataFrame(data=data, schema=columns)

df.show()

+----+-----+
|name|value|
+----+-----+
|  Pi|3.216|
+----+-----+



In [6]:
outpath = '/user/dominospark/mypi/'
df.write.csv(outpath)

AttributeError: 'DataFrame' object has no attribute 'read'

In [8]:
df_load = sparkSession.read.csv(outpath, schema=columns)
df_load.show()

+----+-----+
|name|value|
+----+-----+
|  Pi|3.216|
+----+-----+



In [9]:
sc.stop()

In [10]:
!hdfs dfs -ls /user/dominospark/mypi/

Found 3 items
-rw-r--r--   1 dominospark dominospark          0 2021-08-11 19:27 /user/dominospark/mypi/_SUCCESS
-rw-r--r--   1 dominospark dominospark          0 2021-08-11 19:27 /user/dominospark/mypi/part-00000-b7049172-6fe3-4cfc-9328-5564b309c7b8-c000.csv
-rw-r--r--   1 dominospark dominospark          9 2021-08-11 19:27 /user/dominospark/mypi/part-00003-b7049172-6fe3-4cfc-9328-5564b309c7b8-c000.csv
